In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
anime_data = pd.read_csv('dataset/anime.csv')
anime_data = anime_data[['MAL_ID', 'Name','Genres','Completed']]

In [3]:
# Key: MAL_ID, Value: [genre1, genre2,...]
anime_dict = dict()

anime_genre = dict()

for index, row in anime_data.iterrows():
    mal_id = row[0]
    genres_list = row[2].split(", ")
    
    for genre in genres_list:
        anime_genre[genre] = 1
    anime_dict[mal_id] = genres_list

total_genres = []
for key in anime_genre.keys():
    total_genres.append(key)

total_genres.sort()

In [4]:
top_users = 3000
top_animes = 1000

In [5]:
user_rating = pd.read_csv('rating_complete.csv')
user_ranking = user_rating.groupby(['user_id']).count().sort_values('anime_id',ascending=False)
ranking = user_ranking[0:top_users]
ranking_for_testing = user_ranking[top_users:top_users + 500]
# key: user_id, value:{mal_id:score}
user_anime_dict = dict()

In [6]:
def caluculate_genre_avg_score_for_user(user_id, test_flag=False):
    # Key: genre, Value: count of this genre
    genre_count = dict()
    # Key: genre, Value: score of this genre
    genre_score = dict()
    
    if not test_flag:
        current_user = dict()

    
    for genre in total_genres:
        genre_count[genre] = 0
        genre_score[genre] = 0
    
    piece = user_rating.loc[user_rating['user_id'] == user_id]
    
    for index, row in piece.iterrows():
        mal_id = row["anime_id"]
        score = row["rating"]
        
        if not test_flag:
            current_user[mal_id] = score
        
        mal_genre = anime_dict[mal_id]
        
        for g in mal_genre:
            genre_count[g] = genre_count[g] + 1
            genre_score[g] = genre_score[g] + score
    
    result_avg_score = []
    result_count = []
    for genre in total_genres:
        result_count.append(genre_count[genre])
        if genre_count[genre] == 0:
            result_avg_score.append(0)
            continue
        result_avg_score.append(genre_score[genre] / genre_count[genre])
    
    if not test_flag:
        user_anime_dict[user_id] = current_user
    
    
    return result_avg_score, result_count

In [7]:
# Key: user id, value: avg scores, order the same as total_genres
user_genre_score_dict = dict()
user_genre_score_dict_test = dict()

# Key: user id, value: genre count, order the same as total_genres
user_genre_count_dict = dict()
user_genre_count_dict_test = dict()

for index, row in ranking.iterrows():
    user_id  = index
    
    result_avg_score, result_count = caluculate_genre_avg_score_for_user(user_id)
    user_genre_score_dict[user_id] = result_avg_score
    user_genre_count_dict[user_id] = result_count

for index, row in ranking_for_testing.iterrows():
    user_id  = index
    
    result_avg_score, result_count = caluculate_genre_avg_score_for_user(user_id,True)
    user_genre_score_dict_test[user_id] = result_avg_score
    user_genre_count_dict_test[user_id] = result_count

In [8]:
path = "dataset/users_genres_rating.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["user_id"] + total_genres
    cw.writerow(header)
    
    for user_id in user_genre_score_dict.keys():
        cw.writerow([user_id] + user_genre_score_dict[user_id])
    
    f.close()

In [9]:
path = "dataset/users_genres_rating_test.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["user_id"] + total_genres
    cw.writerow(header)
    
    for user_id in user_genre_score_dict_test.keys():
        cw.writerow([user_id] + user_genre_score_dict_test[user_id])
    
    f.close()

In [10]:
path = "dataset/users_genres_count.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["user_id"] + total_genres
    cw.writerow(header)
    
    for user_id in user_genre_count_dict.keys():
        cw.writerow([user_id] + user_genre_count_dict[user_id])
    
    f.close()

In [11]:
path = "dataset/users_genres_count_test.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["user_id"] + total_genres
    cw.writerow(header)
    
    for user_id in user_genre_count_dict_test.keys():
        cw.writerow([user_id] + user_genre_count_dict_test[user_id])
    
    f.close()

In [12]:
def get_user_completed_anime(mal_id):
    result = []
    
    for i, r in ranking.iterrows():
        uid = i
        
        if mal_id in user_anime_dict[uid]:
            result.append(str(uid) + "|" + str(user_anime_dict[uid][mal_id]))
            
    return result

In [13]:
# Key: mal_id, Value: list of (user id|score)
anime_user = dict()


animes = anime_data.sort_values('Completed', ascending=False)[0:top_animes]
for index, row in animes.iterrows():
    
    mal_id = row["MAL_ID"]
    
    result = get_user_completed_anime(mal_id)
    
    anime_user[mal_id] = result

In [14]:
path = "dataset/top_animes_complete.csv"
with open(path, "w") as f:
    cw = csv.writer(f)
    header = ["MAL_ID", "completed_user_ids", "genres"]
    cw.writerow(header)
    
    for mal_id in anime_user.keys():
        cw.writerow([mal_id, ", ".join(anime_user[mal_id]), ", ".join(anime_dict[mal_id])])
    
    f.close()